In [49]:
import urllib, sys
from bs4 import BeautifulSoup
import re
from collections import Counter
import time
import csv
import pandas as pd
from urllib.error import HTTPError


In [50]:
tickerList = pd.read_excel('../CallTranscript_DownloadTemplate.xlsx')

In [41]:
# wordlist = ['transcript']
# wordlist.extend([str(tickerList['Year'][0]), str(tickerList['Quarter'][0])])
# wordlist

In [51]:
wordlist = ['transcript', '2019', 'q1']

In [52]:
symbol_list = list(tickerList['Ticker'])

In [68]:
# symbol_list= ['ETN', 'FB']

In [53]:
def scrape_data_df(symbol_list,wordlist):
    
#''GE','PEP', 'MHCP',STT
    #symbol_list=['GOOG', 'GE','ETN','PEP','KHC','FB','MCHP','SLAB','JPM','STT']

    #creating a list to store the data for different symbols
    temp_list = []
   

    for symbol in symbol_list:

        site='http://seekingalpha.com/symbol/'+symbol+'/earnings/transcripts'
            #     print(symbol)
        hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                   'Accept-Encoding': 'none',
                   'Accept-Language': 'en-US,en;q=0.8',
                   'Connection': 'keep-alive'} 
        
        try:
            
            pageReq=urllib.request.Request(site,headers=hdr) 
            page=urllib.request.urlopen(pageReq).read()
            soup=BeautifulSoup(page,'lxml')

            for link in soup.find_all('a'):

                x=link.get('href')
                if isinstance(x,str):

                            # Parameterization Pending
                            #wordlist=['transcript','2018','q3']

                    if all(x.find(s)>=0 for s in wordlist):
                        parse_site='http://seekingalpha.com/'+x
                        parse_site='http://seekingalpha.com/'+x+'?part=single'
                        print('Extracting Call Transcripts from: ' + symbol)
                        print(parse_site)

                                # Open the URL as Browser, not as python urllib
                        pageAct=urllib.request.Request(parse_site,headers=hdr) 
                        infileAct=urllib.request.urlopen(pageAct).read()

                        soup = BeautifulSoup(infileAct, 'html5lib') 

                                #creating an empty list
                        text = []

                        try:
                            #saving the scraped data in a list for all the symbols    
                            text.append(soup.find_all('div', attrs = {'id':'a-body'}))

                        except HTTPError as e:
                                print('Error Code', e.code)
                                
                        ######################################################################
                        
#                         try:
#                         # Get content from article body only
#                             text_tscript = soup.findAll('div', attrs = {'id':'a-body'})
#                         # print(text)

#                             for p in text_tscript:
#                             # Extract all the text from within the p tags
# #                             print(p.text)
#                                 transcriptText += p.text_tscript
                        
#                         except HTTPError as e:
#                             print('Error Code', e.code)
#                         transcriptText.replace('\n', ' ')

                               #looping through data for different symbols in the 'text' list
                        for elements in text:

                                    #creating an empty dictionary
                            df_dict = {}

                                    #appending the data of different symbols to dictionary
                            df_dict['Ticker'] = symbol
                            df_dict['Year'] = wordlist[1]
                            df_dict['Quarter'] = wordlist[2]
                            df_dict['PublishedTime'] = "".join([time.text for time in soup.find_all("time", itemprop="datePublished")])
                            df_dict['summary'] = "".join([texts.text for texts in soup.find_all("p")])
                            df_dict['title'] = "".join([title.text for title in soup.find_all("title")])
                            df_dict['CallTranscriptID'] = 'CT_' +symbol + '_' + wordlist[1] + '_' + wordlist[2]
                            

                                    #converting the dictionary to list 
                            temp_list.append(df_dict)
                            
        except HTTPError as e:
            print('Error code:',e.code)
        time.sleep(150)

            #saving the list as a dataframe               
    df2= pd.DataFrame(temp_list)

            #coverting the dataframe output to a csv file
#     df2.to_csv("output.csv", index=False)
#     df2.to_excel('test.xlsx', index=False, header=True)
    return df2


            #increase the time depending on the symbols we are downloading the data for    
    #return df2

In [54]:
# ,'ETN','PEP','KHC','FB','MCHP','SLAB','JPM','STT'
all_data = scrape_data_df(symbol_list,wordlist)


Extracting Call Transcripts from: AXP
http://seekingalpha.com//article/4255382-american-express-company-axp-ceo-stephen-squeri-q1-2019-results-earnings-call-transcript?part=single
Extracting Call Transcripts from: VZ
http://seekingalpha.com//article/4256039-verizon-communications-inc-vz-ceo-hans-vestberg-q1-2019-results-earnings-call-transcript?part=single
Extracting Call Transcripts from: BA
http://seekingalpha.com//article/4256364-boeing-co-ba-ceo-dennis-muilenburg-q1-2019-results-earnings-call-transcript?part=single
Extracting Call Transcripts from: KO
http://seekingalpha.com//article/4256009-coca-cola-company-ko-ceo-james-quincey-q1-2019-results-earnings-call-transcript?part=single
Extracting Call Transcripts from: DIS
http://seekingalpha.com//article/4238411-walt-disney-company-dis-ceo-robert-iger-q1-2019-results-earnings-call-transcript?part=single
Extracting Call Transcripts from: PSX
http://seekingalpha.com//article/4258314-phillips-66-psx-ceo-greg-garland-q1-2019-results-earni

In [55]:
all_data

,CallTranscriptID,PublishedTime,Quarter,Ticker,Year,summary,title
0,CT_AXP_2019_q1,"Apr. 18, 2019 8:15 PM ET",q1,AXP,2019,American Express Co (NYSE:AXP) Q1 2019 Earning...,American Express Company (AXP) CEO Stephen Squ...
1,CT_VZ_2019_q1,"Apr. 23, 2019 3:21 PM ET",q1,VZ,2019,Verizon Communications Inc. (NYSE:VZ) Q1 2019 ...,Verizon Communications Inc. (VZ) CEO Hans Vest...
2,CT_BA_2019_q1,"Apr. 24, 2019 2:58 PM ET",q1,BA,2019,The Boeing Company (NYSE:BA) Q1 2019 Earnings ...,Boeing Co (BA) CEO Dennis Muilenburg on Q1 201...
3,CT_KO_2019_q1,"Apr. 23, 2019 2:24 PM ET",q1,KO,2019,The Coca-Cola Company (NYSE:KO) Q1 2019 Earnin...,The Coca-Cola Company (KO) CEO James Quincey o...
4,CT_DIS_2019_q1,"Feb. 5, 2019 10:01 PM ET",q1,DIS,2019,The Walt Disney Company (NYSE:DIS) Q1 2019 Ear...,The Walt Disney Company (DIS) CEO Robert Iger ...
5,CT_PSX_2019_q1,"Apr. 30, 2019 10:12 PM ET",q1,PSX,2019,Phillips 66 (NYSE:PSX) Q1 2019 Earnings Confer...,Phillips 66 (PSX) CEO Greg Garland on Q1 2019 ...
6,CT_HD_2019_q1,"May 21, 2019 3:00 PM ET",q1,HD,2019,"The Home Depot, Inc. (NYSE:HD) Q1 2019 Earning...","The Home Depot, Inc. (HD) CEO Craig Menear on ..."
7,CT_MCD_2019_q1,"Apr. 30, 2019 6:02 PM ET",q1,MCD,2019,McDonald's Corporation (NYSE:MCD) Q1 2019 Earn...,McDonald's Corporation (MCD) CEO Steve Easterb...


In [56]:
def append_df_to_excel(df, excel_path):
    df_excel = pd.read_excel(excel_path)
    result = pd.concat([df_excel, df], ignore_index=True)
    result.to_excel(excel_path, index=False)

In [57]:
import pandas as pd

append_df_to_excel(all_data, 'test.xlsx')